In [1]:
import torch
from fast_bert.data_cls import BertDataBunch
from fast_bert.learner_cls import BertLearner
from fast_bert.data_lm import BertLMDataBunch
from fast_bert.learner_lm import BertLMLearner
from fast_bert.metrics import fbeta, roc_auc
from fast_bert.prediction import BertClassificationPredictor
from pathlib import Path
import pandas as pd
import logging


logger = logging.getLogger()
device_cuda = torch.device("cuda")

In [2]:
DATA_PATH = Path('./data/')
LOG_PATH = Path('./logs/')
MODEL_PATH = Path('./model/')
LABEL_PATH = Path('./labels/')

In [3]:
df = pd.read_csv('./data/labeled_data.csv')

In [4]:
val_set = df.sample(frac=0.2, replace=False, random_state=42)
train_set = df.drop(index = val_set.index)
print('Nombre de commentaires dans le val_set:',len(val_set))
print('Nombre de commentaires dans le train_set:', len(train_set))
val_set.to_csv('./data/val_set.csv')
train_set.to_csv('./data/train_set.csv')

Nombre de commentaires dans le val_set: 65
Nombre de commentaires dans le train_set: 258


In [5]:
labels = df.columns[2:].to_list()
with open('./labels/labels.txt', 'w') as f:
    for i in labels:
        f.write(i + "\n")

In [6]:
df_texts = pd.read_csv('./data/raw_data.csv')
all_texts = df_texts['caption'].to_list()
print('Nombre de commentaires:', len(all_texts))

Nombre de commentaires: 3899


### Création de LMDataBunch

In [7]:
databunch_lm = BertLMDataBunch.from_raw_corpus(
                    data_dir=DATA_PATH,
                    text_list=all_texts,
                    tokenizer='camembert-base',
                    batch_size_per_gpu=4,
                    max_seq_length=512,
                    multi_gpu=False,
                    model_type='camembert-base',
                    logger=logger)

### Création de LMLearner

In [8]:
lm_learner = BertLMLearner.from_pretrained_model(
                            dataBunch=databunch_lm,
                            pretrained_path='camembert-base',
                            output_dir=MODEL_PATH,
                            metrics=[],
                            device=device_cuda,
                            logger=logger,
                            multi_gpu=False,
                            logging_steps=50,
                            fp16_opt_level="O2")

In [9]:
lm_learner.fit(epochs=30,
            lr=1e-4,
            validate=True,
            schedule_type="warmup_cosine",
            optimizer_type="adamw")

Selected optimization level O2:  FP16 training with FP32 batchnorm and FP32 master weights.

Defaults for this optimization level are:
enabled                : True
opt_level              : O2
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : True
master_weights         : True
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O2
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : True
master_weights         : True
loss_scale             : dynamic


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


/usr/local/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


/usr/local/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0


(2130, 1.5239922608568075)

In [10]:
lm_learner.validate()

{'loss': 0.19354248046875, 'perplexity': 1.2135409116744995}

In [12]:
lm_learner.save_model()

### Création de databunch pour la classification

In [13]:
databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
                          tokenizer='camembert-base',
                          train_file='train_set.csv',
                          val_file='val_set.csv',
                          label_file='labels.txt',
                          text_col='review',
                          label_col=['cadre/atmosphère','probleme technique',"temps d'attente",'accueil/relation commerciale'],
                          batch_size_per_gpu=4,
                          max_seq_length=512,
                          multi_gpu=False,
                          multi_label=True,
                          model_type='camembert-base')

### Création de Learner

In [14]:
metrics = [{'name': 'fbeta', 'function': fbeta}, {'name': 'roc_auc', 'function': roc_auc}]
OUTPUT_DIR = Path('./finetuned_model')
WGTS_PATH = Path('model/model_out/pytorch_model.bin')

In [15]:
cl_learner = BertLearner.from_pretrained_model(
                        databunch,
                        pretrained_path='model/model_out',
                        metrics=metrics,
                        device=device_cuda,
                        logger=logger,
                        output_dir=OUTPUT_DIR,
                        finetuned_wgts_path=WGTS_PATH,
                        warmup_steps=300,
                        multi_gpu=False,
                        multi_label=True,
                        is_fp16=True,
                        logging_steps=50)

model/model_out
<class 'str'>


In [ ]:
cl_learner.fit(epochs=30,
            lr=9e-5,
            validate=True,
            schedule_type="warmup_cosine",
            optimizer_type="adamw")

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 2048.0


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 1024.0


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 512.0


In [ ]:
cl_learner.validate()

In [ ]:
cl_learner.save_model()

### Prédictions

In [30]:
predictor = BertClassificationPredictor(
                model_path='finetuned_model/model_out',
                label_path='labels/',
                multi_label=True,
                model_type='camembert-base',
                do_lower_case=False)

finetuned_model/model_out
<class 'str'>


In [44]:
ex1 = "Très mauvais service je vais chez ma mère le weekend et la TV ne fonctionne pas et très mal tous les weekends j appel pour essayer de résoudre le problème dise que ça viens de la prise électrique j ai un ami qui fait les réparations pour les trois fournisseur et il me dit que free a plein. De problème en ce moment même pas capable de nous le dire appele trois fois ce weekend ok opérateur a pu me mettre la TV on bout de 5 minutes ça raccroche et le dernier ma couper la conversation"
ex2 = "La fibre soit disant bonne est le pire du marché. Elle ne marche que en LAN et pour couronner le tout ils nous prennent pour des ânes quand il y a un soucis il nous demande de faire des Cheats codes avec la télécommande.Service qui nous donne juste envie de vomir quoique la box (télé) est bonne.Je ne vous recommande pas Free."
ex3 = "Services deplorables. Depuis 6 mois le reseau est instable et free ne detecte aucune anomalie. Avoir 6 mega de reception est un exploit on est bien loin des 100 annonces. Impossible d avoir une reception correcte en continu donc coupures internet et television frequentes et quotidiennes. Une etoile est le minimum parcequ il n est pas possilble de n en attribuer aucune."
ex4 = "Je n'ai jamais vu un service client aussi incompétent que celui de free! Free de vrais escrocsJ'ai fait une commande de carte sim le 15 février 2020 qui est restée au statut commande validée et toujours pas expédiée au bout d'une semaine J'ai payé un mois d'abonnement en avance et 10 € de frais de port soit 25,99€. Or je devais recevoir la carte au bout de 5 jours. J'ai appelé 4 fois le service clientèle et à chaque fois j'ai eu une réponse différente, impossible d'avoir une réponse précise sur la date d'expédition de la carte sim. Une personne du service client de free me dit au téléphone qu’il est impossible par free de me donner la carte SIM car la portabilité est impossible. D’après elle, la faute vient de mon ancien opérateur SFR et que FREE ne me remboursera pas tant que je n’ai pas fait une rétractation. Et tant pis pour moi si je ne la fais pas car je ne serai jamais remboursée. Ce dont je ne suis pas d’accord, en effet j’ai fait une commande si Free est dans l’impossibilité de respecter celle-ci, il doit procéder au remboursement, ce ne serait pas à moi de me rétracter en envoyant un courrier par la poste et attendre plusieurs mois avant d’être remboursé. Elle a crié sur moi et a ensuite raccroché!"
ex5 =  "Débit fibre peu performant, avec des bugs fréquents sur le matériel. Régulièrement plusieurs jours sans connexion à la box. Les chaînes TV ne sont pas toutes faciles à trouver (chaîne 17 enregistrée sous le numéro 40, par exemple)."
ex6 = '''Bonjour,

Je fais face à un calvaire que m'imposent les services Bouygues, qui se couvrent derrière des mensonges pour effacer leurs lacunes.

Tout début Août, j'ai passé commande pour une offre fibre Bbox Ultym.
Un premier technicien est venu. Il est parvenu à me connecter (en arrachant les fibres de tous mes voisins de l'étage!! Mais bon, c'est un prestataire, pas Bouygues). J'ai pris un jour de congé pour ça.
Plusieurs jours plus tard, ma ligne n'est toujours pas activée, et je n'ai pas accès à l'ensemble de mes services. J'ai cependant internet, et tout le reste fonctionne.
Je contacte le service client par téléphone, qui me dit qu'ils ne trouvent pas ma ligne et qu'un deuxième rendez vous doit être pris avec un technicien. Je reprends un jour de congé.
Le technicien passe, et me dit, je cite "ils font chier chez Bouygues. Vous n'êtes pas le premier. On leur donne les infos de votre fibre, mais ils ne nous croient pas car dans leur base de données ça ne correspond pas à votre adresse. C'est juste que votre immeuble a été fibré n'importe comment, mais on vient à chaque fois pour ça. Je vous préviens, ça va être long avec eux..."
Il repart sans rien avoir changé, car tout était bon.
J'apprendrais plus tard que les services Bouygues auront noté dans mon dossier "refus de l'installation de la part du client" (??!!)

Fin Août. J'ai toujours internet. Ma ligne n'est pas activée. Je ne comprends pas et me dit qu'ils se réveilleront bien un jour.
En effet, le 29 Août, je reçois un mail "nous sommes contraint d'annuler votre commande SUITE A UN PROBLEME TECHNIQUE". Je rage!
Mais je réalise que j'ai toujours internet. Je gère une équipe et beaucoup de projets, j'ai d'autres choses à faire. Je me pencherai plus tard sur la question.

Le 24 Septembre, je reçois un appel en numéro masqué à 19h. Je ne peux répondre. Un message est laissé sur ma messagerie : "Bonjour, Radia du service raccordement de Bouygues Telecom. Je vous rappelle afin de reprendre un rendez vous avec vous. Je n'ai pas pu vous joindre, je suis contrainte d'annuler votre commande. Au revoir". Pardon ?! Pas de rappel, pas de "vous pouvez me recontacter au...", juste une tentative et POUF, on annule tout!
Je n'ai alors plus internet.

Je rappelle le lendemain, le 25 Septembre, je passe 30 min au téléphone avec une conseillère, sur mon temps de travail, pour expliquer la situation. Elle même est dépitée de cette attitude. Elle me dit qu'elle relance le tout, rendez vous pris le 4 Octobre avec un technicien (encore!!!) pour "activer ma ligne" (qui est active. C'est juste Bouygues qui n'a pas fait le nécessaire!!). Elle m'informe que je n'ai plus à restituer ma clé 4G et que je peux la réutiliser en attendant. Elle m'accorde un (tout petit) 3€ par mois de réduction pour les multiples gênes occasionnées.

Le 26 Septembre, je constate sur internet qu'on me dit toujours de restituer ma clé 4G. Cette dernière ne fonctionne toujours pas.
Le 27 Septembre, je rappelle le service client pour demander ce qu'il se passe et la réactivation de celle-ci.

Et alors là... coup de théâtre. "Ah mais monsieur, je vois que votre commande a été annulée le 26 Septembre, hier."
Pardon ?!
"C'est écrit dans votre dossier monsieur. "Le 26 Septembre, annulation de la commande à la demande du client""
Hein ?!?!
"Je vois que vous avez refusé l'installation du technicien en Aout, puis que vous avez souhaité annuler votre commande fin Aout"
Ah! Donc "suite à un problème technique" se traduit dans les dossiers clients "suite à la demande du client" ?! C'est bien, ça, pour avoir de belles statistiques en termes de relation client non? Autant mentir sur toutes les stats, comme ça, on va paraitre super performants les gars!! Et puis on va dire que le client a annulé sa commande quand on a tout annulé sans le prévenir hein, comme ça, pas de soucis!
Je réfute toutes ces infos, en expliquant que j'ai un rendez vous le 4 Octobre et que j'ai les messages de confirmation de "re souscription" dans ma messagerie bouygues. "C'est écrit que vous avez refusé le rendez vous et que vous avez annulé hier monsieur".
Je ne me suis même pas connecté le 26 Septembre !!
Je demande comment c'est possible "je ne sais pas monsieur"
Je demande à revenir en arrière "c'est pas possible monsieur. Il faut refaire une souscription depuis le début"
Et rebelotte!!!

On refait le tout. Rendez vous avec un technicien le 14 OCTOBRE. Pas internet entre temps.
Je demande quant à la clé 4G, du coup. "C'est écrit que vous n'êtes pas éligible à la clé 4G monsieur, je suis désolé. Il faut aller en boutique pour régler ça"
Je ne suis pas éligible ?!? Donc Bouygues se fout littéralement de moi depuis presque 3 mois, me coupe ma ligne sans me prévenir ni mon accord, et, bonne poire que je suis, quand je reviens quand même, je n'ai plus droit à cette clé 4G ?
On traite bien ses clients chez Bouygues, bravo!

Je ne commenterais pas plus que ça le conseiller qui expédie la conversation pour rapidement y mettre fin avant que j'ai fini de parler...

J'en ai RAS LE BOL. On se croirait au cirque pinder. Entre incompétence, problèmes techniques variés, totale incompréhension de qui a fait quoi et pourquoi, et mensonges scandaleux en pagaille pour se couvrir, on se demande où on est et surtout pourquoi ne pas aller chez la concurrence de suite.

Je souhaites donc escalader ce litige au service réclamation et qu'une solution digne de ce nom soit apportée dans un délai très court (comprenez, avant le 14 Octobre).
Je demande aussi des explications honnêtes et très claires sur qui s'est permis de dire que j'avais demandé à annuler ma commande et pourquoi.
Il va aussi falloir me donner une très bonne raison de ne pas couper court à tout ce foutoir et partir à la concurrence, à peine plus chère que vous, et accordant 5€/mois pour les clients venus de la concurrence (rendant Orange moins cher que vous). Ce n'est pas 3€/mois qui feront la différence.

J'espère que vous comprendrez pourquoi je suis tellement remonté après tout ça, et que nous trouverons une solution amiable. Je n'ai aucun problème à faire appel à un médiateur quitte à escalader encore plus haut ce genre de litiges graves. UFC que choisir, dont je suis membre, me l'a déjà conseillé à ce sujet. Mais je pense qu'une solution à l'amiable est préférable, moins chronophage pour chacun.

J'attends votre retour avec impatience,

Merci de votre compréhension.
'''
ex7 = '''Bonjour CHARLES A.,

Eh oui ! il faut bien se rendre à l'évidence, bouygues télécom (b.t.) c'est beaucoup d'incompétence pour vous sortir de l'embarras rapidement ET SURTOUT un gros foutage de g.....vis à vis de leurs Clients.

bouygues télécom (je ne peux plus mettre de majuscules !) ne devrait s'occuper que du BTP car la COMMUNICATION c'est vraiment pas fait pour eux, ça n'est pas leur truc !

En ce qui me concerne, j'ai une "installation" fibre Must ( forfait de 12 mois) qui a commencé le 12 septembre dernier par un de leur "prestataire"...
Revenu le lendemain ce "prestataire" (je ne veux pas le traiter de c....d) à OSER me laisser une installation en SUSPEND "provisoire" avec ma box dans le garage... POSEE en équilibre SUR MON VELO ! Eh oui on a sans doute pas encore tout vu ...on arrête pas le progrès !
Lui ayant "FAIT SON TRAVAIL"??? la box ne fonctionnant pas dans mon salon il fallait que je me débrouille avec b.t.
Cette "installation" me prive depuis bientôt 3 semaines de LIGNE FIXE .

Après une centaine d'appels au "service ???" Client b.t. qui fait "REMONTER" mon dossier et s'entête de me transférer vers le "service ???" Client darty - TOUT AUSSI NUL ET INOPERANT que le premier cité - car incapable de me donner une date d'intervention...au bout de 5 jours je me suis décidé d'expédier à b.t. un courrier avec AR et Mise en Demeure de:
soit résilier SANS FRAIS mon contrat fibre,
soit sous 10 jours me remettre mon ADSL en service, ds le prolongement et aux conditions de mon ancien contrat existant à la date du 12/09 et pour lequel je n'était plus engagé...
Car entretemps un "expert" b.t. avait signalé que "l'installation" fibre réalisée n'était "ABSOLUMENT PAS CONSEILLEE" ... Ah bon !!! Vous en êtes sûr ?

Le 24 septembre, trois jours après avoir reçu ce courrier, b.t. a daigné m'appeler...C'était bien la 1ère fois en 12 jours !...
On m'a demandé "quel était le problème ???" et ce que je désirais.
On m'a proposé un retour à une installation VDSL, ce que j'ai accepté pour sortir rapidement de cette situation... même si le procédé me semble juridiquement douteux :
Cette nouvelle installation était SOUMISE A UN FORFAIT DE 12MOIS car elle venait en remplacement de mon "installation" fibre.
Bizarre ! b.t. s'étant planté magistralement et ne pouvant pas m'installer la fibre dans de bonnes conditions de fonctionnement m'obligeait donc à accepter un nouveau forfait de 12 mois pour le VDSL !
Voilà les pratiques de b.t. !
Bref ! France Télécom (orange) devait me rappeler après 48 à 72 heures pour m'annoncer une date d'intervention.
Car il faut préciser ici que chez b.t. , le traitement d'une information demande énormément de temps !...Beaucoup plus lentement que pour vous informer de la "signature" de votre contrat (qui se fait en moins de 10 secondes) ou pour adresser vos factures et débiter votre compte bancaire !
France Télécom m'a rappelé le 26 SEPTEMBRE pour me donner une date d'installation pour LE 25 OCTOBRE ...soit au bout de 6 semaines sans pouvoir profiter de la totalité de mes services (je n'ai toujours pas de ligne fixe !).
J'ai refusé ce RDV et je m'en suis plaint au "service???" Clients de b.t. en leur demandant s'ils ne se foutaient pas de leurs Clients .
Là-dessus excuses, "on fait remonter votre dossier" et France Télécom vous rappellera le Lundi 30 septembre (au plus tard !) pour avancer la date d'intervention...

Le lendemain matin, 27 SEPTEMBRE, je reçois un mail m'annonçant qu'il fallait que je retourne mes équipements VDSL ... PAS ENCORE RECEPTIONNES ! Je devais les recevoir que dans l'après-midi.

Le simple fait de refuser un RDV trop tardif ANNULE VOTRE COMMANDE ...même si le "service ???" Clients devait faire "remonter" votre demande auprès de France T. pour avancer leur installation.
"COMPLETEMENT FOUS CES ROMAINS ! " ( lire Astérix pour ceux qui ne comprennent pas l'humour !)

Donc hier soir 27 SEPTEMBRE RE - CONTRAT VDSL...on doit me rappeler d'ici 48 heures...à 7 JOURS (on ouvre le parachute !) pour l'activation de mon nouvel équipement ...et je devrais alors téléphoner à b.t. pour qu'ils viennent m'installer ce VDSL ...EN REMPLACEMENT DE MON "INSTALLATION" fibre qui est toujours en "SUSPEND" au propre comme au figuré dans mon garage ! Cette installation b.t. me la devais au départ et elle s'est terminée par un échec .
J'en suis nullement responsable et je compte bien que b.t. viendra réparer son "plantage" et me remettre une installation VDSL en état de fonctionnement avec tous mes services accessibles ...TRES RAPIDEMENT...ET GRATUITEMENT cela va sans dire !

Je remarque quand même que pour ce second "contrat" de VDSL, Il est curieux que l'on m'ait dit que cette fois France Télécom (orange) n'avait pas besoin d'intervenir... !!!!????
C'est vrai que ce soit chez b.t. ou chez darty, ils ne sont pas à une contradiction près !
Je ne compte plus les informations contraires des unes aux autres que j'ai reçues depuis 15 jours sans parler des fausses informations données tout de suite démenties par le télé-opérateur suivant !
Pour l'instant j'attends mon second équipement VDSL à venir car le 1er reçu hier et toujours emballé doit être retourné ! Il ne pouvait pas servir ???? QUEL CIRQUE !

Peut-être que je serai contraint de me rétracter de cette offre avant 14 jours SI JE N'OBTIENS PAS UNE DATE d'installation de VDSL AVANT LA FIN DE CE DELAI DE RETRACTATION !

b.t. = organisation "MAMMOUTHEENNE" qui met beaucoup de temps à se bouger et à réagir !

Et on voudrait que les Clients soient ravis !

En conclusion :
"C'est dans le besoin que l'on reconnaît ses vrais amis" (proverbe français)....

En conséquence de cette désastreuse expérience du "service??? Client" b.t. et du manque d'un minimum de respect qu'il me prouve,.....dans 12 mois...
JE ME BARRE !... Je n'ai plus confiance en b.t.

Bon courage CHARLES A.... N'en n'attendez pas trop de b.t. !
'''
predictor.predict("Mauvaise ambiance et j'ai trop attendu")

[("temps d'attente", 0.99951171875),
 ('accueil/relation commerciale', 0.98486328125),
 ('cadre/atmosphère', 0.006877899169921875),
 ('probleme technique', 0.002452850341796875)]

In [ ]:
#TODO : TRY LAMB OPTIMIZER ON ALL THE ABOVE